Strings
===

Problems where text parsing and maniuplation play a considerable role in the problem.

You may note that, along with `string`s, you will often find `Span<char>` and `ReadOnlySpan<char>` littered around or entirely replacing `string`s in the problems. `Span<T>` types represent a strongly-typed (`T`) contiguous memory region that can be sliced into a new `Span<T>` and passed around around the stack, which makes them convenient for performing `string` manipulation without allocation overhead.

## 00 Longest Substring Palindrome in a String
### Description
...

### Solution
The general idea is to scan over the array once, checking for the centers of odd-length and even-length palindromes and expanding them as far out as we can. The bounds of the palindrome is kept instead of the `string` so that we don't have to keep reallocating when we find a longer one.

You could optimize this by, if you have found a palindrome of length `m`, by not checking the (this might be slightly off) last `m` characters since you know anything you find won't be longer than what you have - but the general idea stays the same. 

In [1]:
ReadOnlySpan<char> FindLongestPalindrome(ReadOnlySpan<char> maybePalindrome)
{
    ReadOnlySpan<char> lookingGlass = stackalloc char[3]; // how many characters we scan at once
    var longestPalindromeBounds = (lower: 0, upper: 0); // bounds of longest string
    // so that we don't have to keep reallocating strings every time we find a longer one

    // shift lookingGlass through string, offsetting 1 at a time
    for (int i = 0;
        i < maybePalindrome.Length - lookingGlass.Length + 1;
        i++)
    {
        lookingGlass = maybePalindrome[i..(i + lookingGlass.Length)]; // shift glass

        // 2-ple palindrome
        if (lookingGlass[0] == lookingGlass[1])
        {
            TryFindLongestPalindrome((i, i + 1), maybePalindrome);
        }
        // 3-ple palindrome
        else if (lookingGlass[0] == lookingGlass[2])
        {
            TryFindLongestPalindrome((i, i + 2), maybePalindrome);
        }
    }

    // last 2-ple case gets skipped
    if (maybePalindrome.Length > 1
        && longestPalindromeBounds.upper - longestPalindromeBounds.lower < 2
        && lookingGlass[1] == lookingGlass[2])
    {
        longestPalindromeBounds = (maybePalindrome.Length - 2, maybePalindrome.Length);
    }

    void TryFindLongestPalindrome((int lower, int upper) bounds, ReadOnlySpan<char> maybePalindrome)
    {
        // get number of times palindrome could be expanded
        int len = GetCountOfPalindromeExpansion(bounds, maybePalindrome);
        // expand the bounds by larger size, adding 1 to capture ending bound as inclusive
        bounds = (bounds.lower - len, bounds.upper + len + 1);

        // if string longer
        if (bounds.upper - bounds.lower > longestPalindromeBounds.upper - longestPalindromeBounds.lower)
            longestPalindromeBounds = bounds; // set

        /// <summary>
        /// Try to expand a palindrome from its starting point until it is no longer symmetrical, then return the number of expansion steps we made
        /// </summary>
        static int GetCountOfPalindromeExpansion((int, int) start, ReadOnlySpan<char> str)
        {
            var (left, right) = start;

            int len = 0;
            // expand once
            left--;
            right++;

            for (; // increase length as long as symmetry is maintained
                left >= 0
                && right < str.Length
                && str[left] == str[right];

                left--, right++, len++) ;

            return len;
        }
    }

    return maybePalindrome[longestPalindromeBounds.lower..longestPalindromeBounds.upper];
}

In [2]:
new List<string>()
{
    "wesleyracecarobama",
    "",
    "abcde",
    "catcacrat",
    "catracecarcacrat",
    "inevitable",
    "llorange",
    "0cn824gnmm"
}
.Select(s => new { Source = s, Final = FindLongestPalindrome(s).ToString() })
.DisplayTable()

Source,Final
wesleyracecarobama,racecar
,
abcde,
catcacrat,cac
catracecarcacrat,racecar
inevitable,
llorange,ll
0cn824gnmm,mm
